### Longest Bitonic Subsequence

The longest bitonic subsequence problem is to find a subsequence of a given sequence in which the subsequence's elements are first sorted in increasing order, then in dereasing order, and the subsequence is as long as possible.

Example:

    Given sequence is [4, 2, 5, 9, 7, 6, 10, 3, 1]
    Longest bitonic subsequence is [4, 5, 9, 7, 6, 3, 1]
 
For sequences sorted in increasing or decreasing order, the output is the same as the input sequence.

Example:

    [1, 2, 3, 4, 5] -> [1, 2, 3, 4, 5]
    [5, 4, 3, 2, 1] -> [5, 4, 3, 2, 1]
    
We already know how to get the longest increasing subsequence from problem 9. We can do the same to get the longest decreasing subsequence. 

Tweak this so that we get the longest increasing subsequence ending at each position and the longest decreasing subsequence starting at each position.

We then can find the max of these two subsequences added together.

In [122]:
seq1 = [4, 2, 5, 9, 7, 6, 10, 3, 1]
seq2 = [12,13,99,21,35,8,3,76,55,42,11,6,27,16,45,482,351,299,645,868,225,131,431,637,886]

In [22]:
"""
the output is how long each subsequence is if we were to stop 
and include the value at the given position
"""
def longestIncreasingLengthMem(arr):
    output = [1]
    for i in range(1, len(arr)):
        maxLength = 0
        for j in range(0, i):
            if arr[j] < arr[i] and output[j] > maxLength:
                maxLength = output[j]
        output.append(maxLength + 1)
    return output

In [23]:
"""
the output is how long each subsequence is if we were to start and 
include the value at the given position
"""

def longestDecreasingLengthMem(arr):
    output = [1]
    m = len(arr)
    for i in range(2, m + 1):
        maxLength = 0
        for j in range(0, i):
            if arr[m - 1 - j] < arr[m - i] and output[j] > maxLength:
                maxLength = output[j]
        output.append(maxLength + 1)
    return output[::-1]

In [125]:
def longestBitonicLength(arr):
    increasing = longestIncreasingLengthMem(arr)
    decreasing = longestDecreasingLengthMem(arr)
    bitonic_lengths = [increasing[i] + decreasing[i] for i in range(len(arr))]
    return max(bitonic_lengths) - 1

In [105]:
%%time
longestBitonicLength(seq1)

[4, 3, 5, 8, 7, 6, 7, 4, 2]
CPU times: user 214 µs, sys: 91 µs, total: 305 µs
Wall time: 260 µs


7

In [126]:
%%time
longestBitonicLength(seq2)

CPU times: user 225 µs, sys: 1 µs, total: 226 µs
Wall time: 231 µs


11

#### If we want the actual sequence, not just the length

We'll have to store the subsequences as we search for the longest increasing and longest decreasing subsequences, not just the lengths.

In [30]:
def longestIncreasing(arr):
    output = [[arr[0]]]
    for i in range(1, len(arr)):
        maxLength = 0
        maxPos = -1
        for j in range(0, i):
            if arr[j] < arr[i] and len(output[j]) > maxLength:
                maxLength = len(output[j])
                maxPos = j
        if maxPos == -1:
            output.append([arr[i]])
        else:
            max_sub = output[maxPos][:]
            max_sub.append(arr[i])
            output.append(max_sub)
    return output

In [67]:
def longestDecreasing(arr):
    m = len(arr)
    output = [[arr[m - 1]]]
    for i in range(2, m + 1):
        maxLength = 0
        maxPos = -1
        for j in range(0, i):
            if arr[m - 1 - j] < arr[m - i] and len(output[j]) > maxLength:
                maxLength = len(output[j])
                maxPos = j
        if maxPos == -1:
            output.append([arr[m - i]])
        else:
            max_sub = output[maxPos][:]
            max_sub.insert(0, arr[m - i])
            output.append(max_sub)
    return output[::-1]
        
    

In [69]:
def longestBitonic(arr):
    increasing = longestIncreasing(arr)
    decreasing = longestDecreasing(arr)
    bitonicLengths = [(len(increasing[i]) + len(decreasing[i]) - 1) for i in range(len(arr))]
    max_bitonic = max(bitonicLengths)
    max_pos = bitonicLengths.index(max_bitonic)
    increasing[max_pos].extend(decreasing[max_pos][1:])
    return increasing[max_pos]

In [118]:
%%time
longestBitonic(seq1)

CPU times: user 51 µs, sys: 12 µs, total: 63 µs
Wall time: 65.1 µs


[4, 5, 9, 7, 6, 3, 1]

In [128]:
%%time
longestBitonic(seq2)

CPU times: user 183 µs, sys: 17 µs, total: 200 µs
Wall time: 203 µs


[12, 13, 21, 35, 42, 45, 482, 351, 299, 225, 131]

#### With classic recursion

With any given value, we have a number of things to decide.

1. Are we increasing? Shall we continue to increase?
2. Can this value be added? Shall we add it?

We try all of our options and call the next iteration, taking the largest value returned.

In [86]:
def bitonic_helper(arr, pos, ap_sub, sub):
    return max(
        bitonic(arr, True, pos, ap_sub),
        bitonic(arr, True, pos, sub),
        bitonic(arr, False, pos, ap_sub),
        bitonic(arr, False, pos, sub)
    )

In [101]:
def bitonic(arr, inc=True, pos=0, sub=None):
    if sub == None:
        sub = []
    if pos >= len(arr):
        return len(sub)
    if pos == 0:
        ap_sub = sub[:]
        ap_sub.append(arr[0])
        return bitonic_helper(arr, pos+1, ap_sub, sub)
    if inc == True:
        if len(sub) == 0 or arr[pos] > sub[-1]:
            ap_sub = sub[:]
            ap_sub.append(arr[pos])
            return bitonic_helper(arr, pos+1, ap_sub, sub)
        return max(
            bitonic(arr, True, pos+1, sub),
            bitonic(arr, False, pos+1, sub)
        )
    else:
        if len(sub) == 0 or arr[pos] < sub[-1]:
            ap_sub = sub[:]
            ap_sub.append(arr[pos])
            return max(
                bitonic(arr, False, pos+1, sub),
                bitonic(arr, False, pos+1, ap_sub)
            )
        return bitonic(arr, False, pos+1, sub)
    
    

In [104]:
%%time
bitonic(seq1)

CPU times: user 857 µs, sys: 0 ns, total: 857 µs
Wall time: 862 µs


7

In [124]:
%%time
bitonic(seq2)

CPU times: user 156 ms, sys: 2.49 ms, total: 158 ms
Wall time: 160 ms


11

In [114]:
def bitonicSequenceHelper(arr, pos, ap_sub, sub):
    inc_ap = bitonicSequence(arr, True, pos, ap_sub)
    inc_sub = bitonicSequence(arr, True, pos, sub)
    dec_ap = bitonicSequence(arr, False, pos, ap_sub)
    dec_sub = bitonicSequence(arr, False, pos, sub)
    sequences = [inc_ap, inc_sub, dec_ap, dec_sub]
    seq = max(sequences, key=len)
    return seq

In [115]:
def bitonicSequence(arr, inc=True, pos=0, sub=None):
    if sub == None:
        sub = []
    if pos >= len(arr):
        return sub
    if pos == 0:
        ap_sub = sub[:]
        ap_sub.append(arr[0])
        return bitonicSequenceHelper(arr, pos+1, ap_sub, sub)
    if inc == True:
        if len(sub) == 0 or arr[pos] > sub[-1]:
            ap_sub = sub[:]
            ap_sub.append(arr[pos])
            return bitonicSequenceHelper(arr, pos+1, ap_sub, sub)
        return max(
            [bitonicSequence(arr, True, pos+1, sub),
            bitonicSequence(arr, False, pos+1, sub)], key=len
        )
    else:
        if len(sub) == 0 or arr[pos] < sub[-1]:
            ap_sub = sub[:]
            ap_sub.append(arr[pos])
            return max(
                [bitonicSequence(arr, False, pos+1, sub),
                bitonicSequence(arr, False, pos+1, ap_sub)], key=len
            )
        return bitonicSequence(arr, False, pos+1, sub)

In [117]:
%%time
bitonicSequence(seq1)

CPU times: user 1.25 ms, sys: 1 µs, total: 1.25 ms
Wall time: 1.25 ms


[4, 5, 9, 7, 6, 3, 1]

In [127]:
%%time
bitonicSequence(seq2)

CPU times: user 170 ms, sys: 2.92 ms, total: 173 ms
Wall time: 173 ms


[12, 13, 21, 35, 42, 45, 482, 645, 868, 225, 131]